# The Differentiable Neural Computer

## The Problem - how do we create more general purpose learning machines?

Neural networks excel at pattern recognition and quick, reactive decision-making, but we are only just 
beginning to build neural networks that can think slowly. that is, deliberate or reason using knowledge.
For example, how could a neural network store memories for facts like the connections in a transport network 
and then logically reason about its pieces of knowledge to answer questions?

![alt text](https://storage.googleapis.com/deepmind-live-cms/images/dnc_figure1.width-1500_Zfxk87k.png "Logo Title Text 1")

this consists of a neural network that can read from and write to an external memory matrix,
analogous to the random-access memory in a conventional computer.

Like a conventional computer, it can use its memory to represent and manipulate complex data structures, 
but, like a neural network, it can learn to do so from data.

DNCs have the capacity to solve complex, structured tasks that are 
inaccessible to neural networks without external read–write memory.

![alt text](https://storage.googleapis.com/deepmind-live-cms/images/dnc_figure2.width-1500_be2TeKT.png "Logo Title Text 1")

[![IMAGE ALT TEXT HERE](http://img.youtube.com/vi/B9U8sI7TcMY/0.jpg)](http://www.youtube.com/watch?v=B9U8sI7TcMYE)



Modern computers separate computation and memory. Computation is performed by a processor, 
which can use an addressable memory to bring operands in and out of play. 

In contrast to computers, the computational and memory resources of artificial neural networks 
are mixed together in the network weights and neuron activity. This is a major liability: 
as the memory demands of a task increase, these networks cannot allocate new storage 
dynam-ically, nor easily learn algorithms that act independently of the values realized 
by the task variables.
    
The whole system is differentiable, and can therefore be trained 
end-to-end with gradient descent, allowing the network to learn 
how to operate and organize the memory in a goal-directed manner.

If the memory can be thought of as the DNC’s RAM, then the network, referred to as the ‘controller’, 
is a differentiable CPU whose operations are learned with gradient descent.



How is it different from its predecessor, the Neural Turing Machine?

basically, more memory access methods than NTM

 DNC extends the NTM addressing the following limitations:

(1) Ensuring that blocks of allocated memory do not overlap and interfere.

(2) Freeing memory that have already been written to.

(3) Handling of non-contiguous memory through temporal links.


the system required hand-crafted input to accomplish its learning and inference. This is not an NLP system where unstructured text is applied at input. 

3 forms of attention for heads
- content lookup
- records transitions between consecutively written locations in an N ×  N temporal link matrix L.
This gives a DNC the native ability to recover sequences in the order in which it wrote them, even
when consecutive writes did not occur in adjacent time-step
- The third form of attention allocates memory for writing. 

Content lookup enables the formation of associative data structures;
temporal links enable sequential retrieval of input sequences;
and allocation provides the write head with unused locations. 

DNC memory modification is fast and can be one-shot, resembling the associative 
long-term potentiation of hippocampal CA3 and CA1 synapses

 Human ‘free recall’ experiments demonstrate the increased probability of 
   item recall in the same order as first pre-sented (temporal links)
    
DeepMind hopes that DNCs provide both a new tool for computer science and a new metaphor for cognitive science
and neuroscience: here is a learning machine that, without prior programming, can organise information
into connected facts and use those facts to solve problems.

In [1]:
import numpy as np
import tensorflow as tf
import os

/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
class DNC:
    def __init__(self, input_size, output_size, seq_len, num_words = 256, word_size = 64, num_heads = 4):
        '''
        Initialize the DNC:
        In this tutorial we are basically using the DNC to understand the mapping between the input
        and output data.
        
        input data: [[0,0], [0,1], [1,0], [1,1]]
        output data: [[1,0], [0,0], [0,0], [0,1]]
        '''
        # define input and output sizes
        self.input_size = input_size
        self.output_size = output_size
        
        # define read and write vectors
        self.num_words = num_words # N
        self.word_size = word_size # W
        
        # define number of read and write heads
        self.num_heads = num_heads # R
        
        # size of output vector from controller
        # the magic numbers are just a type of hyper-parameters
        # set them according to your own use-case, they come from the way we divide our
        # interface vector into read, write, gate and read mode variables
        self.interface_size = num_heads*word_size + 3*word_size + 5*num_heads + 3
        
        # define input size
        # this comes after the flatten the input and concatenate it with the
        # previously read vectors from the memory
        self.nn_input_size = num_heads*word_size + input_size
        
        # define output size
        self.nn_output_size = output_size + self.interface_size
        
        # gaussian normal distribution for both outputs
        # ???
        self.nn_out = tf.truncated_normal([1, self.output_size], stddev = 0.1)
        self.interface_vec = tf.truncated_normal([1, self.interface_size], stddev = 0.1)
        
        # define memory matrix
        self.mem_mat = tf.zeros([num_words, word_size]) # N*W
        
        # define usage vector
        # it tells which part of the memory have been used so far
        self.usage_vec = tf.fill([num_words, 1], 1e-6) # W*1
        
        # define temporal link matrix
        # it tells in which order the locations were written
        self.link_mat = tf.zeros([num_words, num_words]) # N*N
        
        # define precedence weight
        # it tell to the degree which the last weight was written to
        self.precedence_weight = tf.zeros([num_words, 1]) # N*1
        
        # define read and write weight variables
        self.read_weights = tf.fill([num_words, num_heads], 1e-6) # N*R
        self.write_weights = tf.fill([num_words, 1], 1e-6) # N*1
        self.read_vec = tf.fill([num_heads, word_size], 1e-6) # N*W
        
        #######################
        ## Network Variables ##
        #######################
        
        # parameters
        hidden_layer_size = 32
        
        # define placeholders
        self.i_data = tf.placeholder(tf.float32, [seq_len*2, self.input_size], name = 'input_placeholder')
        self.o_data = tf.placeholder(tf.float32, [seq_len*2, self.output_size], name = 'output_placeholder')
        
        # define feedforward network weights
        self.W1 = tf.Variable(tf.truncated_normal([self.nn_input_size, hidden_layer_size], stddev = 0.1),
                              name = 'layer1_weights', dtype = tf.float32)
        self.b1 = tf.Variable(tf.truncated_normal([hidden_layer_size], stddev = 0.1),
                             name = 'layer1_bias', dtype = tf.float32)
        self.W2 = tf.Variable(tf.truncated_normal([hidden_layer_size, self.nn_output_size], stddev = 0.1),
                              name = 'layer2_weights', dtype = tf.float32) 
        self.b2 = tf.Variable(tf.truncated_normal([self.nn_output_size], stddev = 0.1),
                             name = 'layer2_bias', dtype = tf.float32)
        
        # define DNC output weights
        # self.nn_out_weights to convert the output of neural network into proper output
        self.nn_out_weights = tf.Variable(
            tf.truncated_normal([self.nn_output_size, self.output_size],
                                stddev = 0.1),
            name = 'nn_output_weights', dtype = tf.float32)
        # self.interface_weights to convert the output of neural network to proper interface vector
        self.interface_weights = tf.Variable(
            tf.truncated_normal([self.nn_output_size, self.interface_size],
                                stddev = 0.1),
            name = 'interface_weights', dtype = tf.float32)
        #
        self.read_vec_out_weights = tf.Variable(
            tf.truncated_normal([self.num_heads*self.word_size, self.output_size],
                               stddev = 0.1),
            name = 'read_vector_output_weights', dtype = tf.float32)
        
    ##########################
    ## Attention Mechanisms ##
    ##########################
    '''
    In DNC we have three different attention mechanisms:
    1. Content Lookup (Content-Addressing in paper):
        {From NTM paper} For content-addressing, each head (whether employed for reading or
        writing) first produces a key-vector k, that is then compared to each vector in memory by
        a similarity measure. The content-based system produces a normalized weighting based
        on similarity [and a positive key-strength (beta), which can amplify or attenuate the
        precision of the focus.]
    2. Allocation weighting:
        {From DNC paper} To allow controller to free and allocate memory as needed, we developed
        a differentiable analogue to 'free-list' memory scheme, whereby a a list of available
        memory location is maintained by adding to and removig from a linked list.
        {From tutorial} The ‘usage’ of each location is represented as a number between 0 and 1,
        and a weighting that picks out unused locations is delivered to the write head. This is
        independent of the size and contents of the memory, meaning that  DNCs can be trained to
        solve a task using one size of memory and later upgraded to a larger memory without
        retraining
    3. Temporal Linking:
        {From DNC paper} The memory location defined [till now] stores no information about the
        order in which memory locations are written to. However, there are many situation where
        retaining this information is useful: for example, when a sequence inrtuctions must be
        recorded and retrieved in order. We therefore use a temporal link matrix to keep track
        of consecutively modified memory locations.
    '''

    # define content lookup
    def content_lookup(self, key, str):
        # str is 1*1 or 1*R
        # l2 normalization of a vector is the square root of sum of absolute values squared
        norm_mem = tf.nn.l2_normalize(self.mem_mat, 1) # N*W
        norm_key = tf.nn.l2_normalize(key, 0) # 1*W for write, R*W for read
        sim = tf.matmul(norm_mem, norm_key, transpose_b = True) # N*1 for write, N*R for read
        return tf.nn.softmax(sim*str, 0) # N*1 or N*R

    # define allocation weighting
    def allocation_weighting(self):
        # tf.nn.top_k() returns
        # 1.The k largest elements along each last dimensional slice and
        # 2.The indices of values within the last dimension of input
        sorted_usage_vec, free_list = tf.nn.top_k(-1*self.usage_vec, k = self.num_words)
        sorted_usage_vec *= -1
        # tf.cumprod() calculates cumulative product
        # tf.cumprod([a, b, c]) --> [a, a*b, a*b*c]
        # tf.cumprod([a, b, c], exclusive=True) --> [1, a, a * b]
        cumprod = tf.cumprod(sorted_usage_vec, axis = 0, exclusive = True)
        unorder = (1-sorted_usage_vec)*cumprod

        # allocation weight
        alloc_weights = tf.zeros([self.num_words])
        I = tf.constant(np.identity(self.num_words, dtype = np.float32))

        # for each usage vector
        for pos, idx in enumerate(tf.unstack(free_list[0])):
            m = tf.squeeze(tf.slice(I, [idx, 0], [1, -1]))
            alloc_weights += m*unorder[0, pos]

        # allocation weighting for each row in memory
        return tf.reshape(alloc_weights, [self.num_words, 1])

    ###################
    ## Step Function ##
    ###################

    # define the step function
    '''
    This is the function that we call while we are running our session at each iteration the
    controller recieves two inputs that are concatenated, the input vector and the read vector
    from previous time step it also gives two outputs, the output vector and the interface
    vector that defines it's interaction with the memory at the current time step.
    '''
    def step_m(self, input_seq):
        '''print('input_seq:',input_seq)
        print('self.read_vec:', self.read_vec)
        print('reshape',tf.reshape(self.read_vec, [1, self.num_words*self.word_size]))
        print(tf.concat([input_seq, tf.reshape(self.read_vec, [1, self.num_heads*self.word_size])], 1))'''
        # reshape the input
        input_vec_nn = tf.concat([input_seq, tf.reshape(self.read_vec, [1, self.num_heads*self.word_size])], 1)

        # forward propogation
        l1_out = tf.matmul(input_vec_nn, self.W1) + self.b1
        l1_act = tf.nn.tanh(l1_out)
        l2_out = tf.matmul(l1_out, self.W2) + self.b2
        l2_act = tf.nn.tanh(l2_out)

        # output vector, the output of the DNC
        self.nn_out = tf.matmul(l2_act, self.nn_out_weights)

        # interface vector, how to interact with the memory
        self.interface_vec = tf.matmul(l2_act, self.interface_weights)

        # define partition vector
        '''
        We need to get lot of information from the interface vector, which will help us get various
        vectors such as read vectors, write vectors, degree to which locations will be freed
        '''
        p_array = [0]*(self.num_heads * self.word_size) # read keys
        p_array += [1]*(self.num_heads) # read string
        p_array += [2]*(self.word_size) # write key
        p_array += [3] # write string
        p_array += [4]*(self.word_size) # erase vector
        p_array += [5]*(self.word_size) # write vector
        p_array += [6]*(self.num_heads) # free gates
        p_array += [7] # allocation gates
        p_array += [8] # write gates
        p_array += [9]*(self.num_heads*3) # read mode
        partition = tf.constant([p_array])

        # convert interface vector to set of read write vectors
        (read_keys, read_str, write_key, write_str, erase_vec,
         write_vec, free_gates, alloc_gate, write_gate, read_modes) = \
        tf.dynamic_partition(self.interface_vec, partition, 10)

        # read vectors
        read_keys = tf.reshape(read_keys, [self.num_heads, self.word_size]) # R*W
        read_str = 1 + tf.nn.softplus(tf.expand_dims(read_str, 0)) # 1*R

        # write vectors
        write_key = tf.expand_dims(write_key, 0) # 1*W
        write_str = 1 + tf.nn.softplus(tf.expand_dims(write_str, 0)) # 1*1
        erase_vec = tf.nn.sigmoid(tf.expand_dims(erase_vec, 0)) # 1*W
        write_vec = tf.expand_dims(write_vec, 0) # 1*w

        # gates
        # free gates, the degree to which the locations at read head will be freed
        free_gates = tf.nn.sigmoid(tf.expand_dims(free_gates, 0)) # 1*R
        # the fraction of writing that is being allocated in a new location
        alloc_gate = tf.nn.sigmoid(alloc_gate) # 1
        # the amount of information to be written to memory
        write_gate = tf.nn.sigmoid(write_gate) # 1

        # read modes
        # we do a softmax distribution between 3 read modes (backward, forward, lookup)
        # The read heads can use gates called read modes to switch between content lookup 
        # using a read key and reading out locations either forwards or backwards 
        # in the order they were written.
        read_modes = tf.nn.softmax(tf.reshape(read_modes, [3, self.num_heads])) # 3*R

        ## WRITING

        # the memory retention vector tells by how much each location will not be freed
        # by the free gates, helps in determining usage vector
        retention_vec = tf.reduce_prod(1 - free_gates*self.read_weights, reduction_indices = 1)
        self.usage_vec = (self.usage_vec + self.write_weights - \
                          self.usage_vec*self.write_weights) * retention_vec

        # allocation weighting is used to provide new locations for writing
        alloc_weights = self.allocation_weighting()
        write_lookup_weights = self.content_lookup(write_key, write_str)

        # define write weights
        self.write_weights = write_gate*(alloc_gate*alloc_weights + \
                            (1-alloc_gate)*write_lookup_weights)

        # write -> erase -> write to memory
        self.mem_mat = self.mem_mat*(1 - tf.matmul(self.write_weights, erase_vec)) + \
                            tf.matmul(self.write_weights, write_vec)
            
        # temporal link matrix
        nnweight_vec = tf.matmul(self.write_weights, tf.ones([1, self.num_words])) # N*N
        self.link_mat = self.link_mat*(1 - nnweight_vec - tf.transpose(nnweight_vec)) + \
                        tf.matmul(self.write_weights, self.precedence_weight, transpose_b = True)
        self.link_mat *= tf.ones([self.num_words, self.num_words]) - \
                        tf.constant(np.identity(self.num_words, dtype = np.float32))
            
        # update precedence weight
        self.precedence_weight = (1 - tf.reduce_sum(self.write_weights, reduction_indices = 0)) * \
                                self.precedence_weight + self.write_weights
        
        # 3 read modes
        forw_w = read_modes[2] * tf.matmul(self.link_mat, self.read_weights)
        look_w = read_modes[1] * self.content_lookup(read_keys, read_str)
        back_w = read_modes[0] * tf.matmul(self.link_mat, self.read_weights, transpose_a = True)
        
        # initialize read weights
        self.read_weights = forw_w + look_w + back_w
        
        # read vector
        self.read_vec = tf.transpose(tf.matmul(self.mem_mat, self.read_weights, transpose_a = True))
        
        # get final read output
        read_vec_mut = tf.matmul(tf.reshape(self.read_vec, [1, self.num_heads*self.word_size]),
                                self.read_vec_out_weights)
        
        # return the final output
        return self.nn_out + read_vec_mut
    
    # output the list of numbers (one hot encoded) by running step function
    def run(self):
        big_out = []
        for t, seq in enumerate(tf.unstack(self.i_data, axis = 0)):
            seq = tf.expand_dims(seq, 0)
            y = self.step_m(seq)
            big_out.append(y)
        return tf.stack(big_out, axis = 0)

In [3]:
# generate randomly generated input, output sequences
num_seq = 10
seq_len = 6
seq_width = 4
num_epochs = 1000
con = np.random.randint(0, seq_width,size=seq_len)
seq = np.zeros((seq_len, seq_width))
seq[np.arange(seq_len), con] = 1
end = np.asarray([[-1]*seq_width])
zer = np.zeros((seq_len, seq_width))

In [4]:
# final i/o data
final_i_data = np.concatenate((seq, zer), axis = 0)
final_o_data = np.concatenate((zer, seq), axis = 0)

In [7]:
# define compute graph
graph = tf.Graph()

# running the graph
with graph.as_default():
    with tf.Session() as sess:
        # define the DNC
        dnc = DNC(input_size = seq_width,
                 output_size = seq_width,
                 seq_len = seq_len,
                 num_words = 10,
                 word_size = 4,
                 num_heads = 1)
        
        #calculate the predicted output
        output = tf.squeeze(dnc.run())
        #compare prediction to reality, get loss via sigmoid cross entropy
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=output, labels=dnc.o_data))
        #use regularizers for each layer of the controller
        regularizers = (tf.nn.l2_loss(dnc.W1) + tf.nn.l2_loss(dnc.W2) +
                        tf.nn.l2_loss(dnc.b1) + tf.nn.l2_loss(dnc.b2))
        #to help the loss convergence faster
        loss += 5e-4 * regularizers
        #optimize the entire thing (memory + controller) using gradient descent. dope
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

        #initialize input output pairs
        sess.run(tf.global_variables_initializer())
        
        #for each iteration
        for i in range(0, num_epochs+1):
            #feed in each input output pair
            feed_dict = {dnc.i_data: final_i_data, dnc.o_data: final_o_data}
            #make predictions
            l, _, predictions = sess.run([loss, optimizer, output], feed_dict=feed_dict)
            if i%100==0:
                print(i,l)
                
        # print predictions
        print(np.argmax(final_i_data, 1))
        print(np.argmax(final_o_data, 1))
        print(np.argmax(predictions, 1))

0 0.716662
100 0.291819
200 0.139632
300 0.102837
400 0.0829762
500 0.0597146
600 0.034657
700 0.0176897
800 0.01247
900 0.01024
1000 0.00889811
[3 1 1 3 3 0 0 0 0 0 0 0]
[0 0 0 0 0 0 3 1 1 3 3 0]
[2 3 3 3 3 3 3 1 1 3 3 0]
